In [1]:
import numpy as np
import pandas as pd
import sqlite3
from PIL import Image
from pathlib import Path
import argparse
from tqdm import tqdm
import glob
import os
import skimage.io
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from functools import partial, reduce
import time
from sqlalchemy import create_engine

In [2]:
top_dir = '/home/ubuntu/bucket/projects'
proj_dir = '2018_11_20_GeneCpdFollowup'
batch= '2018_11_20_Batch1'
plates = ['BR00100032', 'BR00100037']

outpath = '/home/ubuntu/bucket/projects/2018_11_20_GeneCpdFollowup/workspace/backend/2018_11_20_Batch1/'




In [14]:
class load_data:
    
    def __init__(self,top_dir,proj_dir, batch, plates):
        
        self.top_dir = top_dir
        self.proj_dir = proj_dir
        self.batch = batch
        self.plates = plates
        

    def sqlpath(self):
        path = os.path.join(self.top_dir, self.proj_dir, 'workspace', 'backend', self.batch)
        
        spath = [os.path.join(path, p, p + ".sqlite") for p in self.plates]
        
        return spath
    
    def csvpath(self):
        
        path = os.path.join(self.top_dir, self.proj_dir, 'workspace', 'backend', self.batch)
        
        spath = [os.path.join(path, p, p + "selected_CDK14_ERK5.csv") for p in self.plates]
        
        return spath
    def dmsopath(self):
        
        path = os.path.join(self.top_dir, self.proj_dir, 'workspace', 'backend', self.batch)
        
        spath = [os.path.join(path, p, p + "_selected_wells.csv") for p in self.plates]
        
        return spath
    
    
    
        

path = load_data(top_dir, proj_dir, batch, plates)
sqlpath = path.sqlpath()
csvpath = path.csvpath()[0]
dmsopath = path.dmsopath()
dmsopath = dmsopath[0]
dmsopath

'/home/ubuntu/bucket/projects/2018_11_20_GeneCpdFollowup/workspace/backend/2018_11_20_Batch1/BR00100032/BR00100032_selected_wells.csv'

In [33]:
def sqlite_connect(path):
    
    engine = create_engine(os.path.join('sqlite:///'+path))
    image = pd.read_sql_query("select *from Image", engine)
    cells = pd.read_sql_query("select  * from Cells", engine)
    cyto= pd.read_sql_query("select * from Cytoplasm", engine)
    nuclei= pd.read_sql_query("select * from Nuclei", engine)
    dt = reduce(lambda x,y: pd.merge(x,y, on=["TableNumber", "ImageNumber", "ObjectNumber"], how='left'), [cells, nuclei, cyto])
    df = reduce(lambda x,y: pd.merge(x,y, on=["TableNumber", "ImageNumber"], how='left'), [dt, image])

    return df

In [ ]:
starttime = time.time()
for spath, pl in zip(sqlpath, plates):
    
    df = sqlite_connect(spath)
    selected_wells = ['F04', 'F08', 'F17', 'F21', 'G03', 'G04', 'G07', 'G08', 'G16',
       'G17', 'G20', 'G21', 'H03', 'H04', 'H07', 'H08', 'H16', 'H17',
       'H20', 'H21', 'I05', 'I06', 'I08', 'I09', 'I18', 'I19', 'I21',
       'I22', 'J05', 'J06', 'J08', 'J09', 'J18', 'J19', 'J21', 'J22',
       'K06', 'K09', 'K19', 'K22']
    
    df = df.query("Metadata_Well in @selected_wells")
    df.to_csv(os.path.join(outpath, pl, pl+'selected_FAK.csv'))

finishtime = time.time()
print('That took {} minutes'.format((finishtime - starttime)/60))
    